**Import file named as pressure.py for the Pressure**


In [ ]:
path=r'C:\Users\UWAdmin\Desktop\AIM-Lab-Automation-master\AIM-Lab-Automation-master\codes'
import os
#os.chdir(path)

from pressure import close_valve,open_valve,close_all,current_pressure,gopr
current_pressure()

**Make sure all the valves are closed and check the current pressure**

In [ ]:
close_all()
current_pressure()


**Vaccum the chamber**

In [ ]:
open_valve('vac')

In [ ]:
close_valve('vac')

**Create Initial random dataset**

In [ ]:
##Create the initial random dataset

import numpy as np
import csv
power=[]
time=[]
pressure=[]
gas=[]
print("Enter number of initial samples: ")
n=int(input())
for x in range(n):
    power.append(np.random.randint(150,5000))
    time.append(np.random.randint(500,10000))
    pressure.append(np.random.randint(100,1000))

    
from datetime import date

p=str(date.today())
path=r'C:\Users\UWAdmin\Desktop\AIM-Lab-Automation-master\AIM-Lab-Automation-master\codes'
import os
os.chdir(path)
os.mkdir('Campaign ' + p)
os.chdir('Campaign ' + p)
row=['power','time','pressure','ratio']
with open('dataset.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
        writer.writerows(zip(power,time,pressure))
        
os.chdir(path)

In [ ]:
import os
os.getcwd()

## Run the sections A, B, C, D sequentially. Then the main program is ready to run and conduct experiment.

## (A) Next cells before B are to initiate motion controller.

In [ ]:
#from pymeasure.instruments.newport import ESP300
from pymeasure.instruments import list_resources

import ftd2xx  # Thorlabs MFF101
import ftd2xx.defines as constants
from ftd2xx import listDevices, getDeviceInfoDetail

from time import sleep

In [ ]:
from newportxps import NewportXPS
from newportxps.XPS_C8_drivers import XPS, XPSException
import pysftp

# from ophyd import EpicsSignal
from collections import OrderedDict
import yaqc
import os
import platform

In [ ]:
xpsd_remoteip = '192.168.254.254'
# xpsd_hostip ='192.168.0.254'
hostname = 'XPS-1b81'

xps = NewportXPS(hostname)
print(xps.status_report())

In [ ]:
print(xps.status_report())

In [ ]:
xps.stages

In [ ]:
xps.kill_group('XYZ')
xps.initialize_allgroups()
xps.home_allgroups()

In [ ]:
def pos_all():
    for sname, _ in xps.stages.items():
        print('{}: {}'.format(sname, xps.get_stage_position(sname)))

In [ ]:
pos_all()

## (B)

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## (C) The next bunch of code should be run to initialize the Spectrometer (PrincetonInstruments Lightfield Software) and loads the curve fitting code

In [ ]:
from spectra import capture_photo,ration
capture_photo("start",2) #it starts the lightfield software
import pandas as pd


## (D) Import the flipper mirror and Laser Power controlling package**

In [ ]:
from flipper import mirror
#mirror('off')
from pymeasure.instruments.lighthousephotonics import Sprout
laser_power = Sprout('COM8')
laser_power.adapter.connection.baud_rate = 19200
laser_power.adapter.connection.read_termination = '\r'
laser_power.adapter.connection.write_termination = '\r'
laser_power.power=0.01

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

**Before beginning the experiment, change the current directory to the current campaign folder**

In [ ]:
    
from datetime import date
pp=str(date.today())
os.chdir('Campaign ' + pp)

## Main Program to begin the experiment
**Here the main experiment begins**

In [ ]:

import pandas as pd
laser_power.power=0.01
import time
import os

df=pd.read_csv("dataset.csv")
df.head()
power_of_file=df["power"]
time_of_file=df["time"]
pressure_of_file=df['pressure']


#initialize position
print("Initializing starts\n")

pos_x=[]
pos_y=[]
import numpy as np
p=2
p2=2
print("Enter the (y1,x1) coordinate")
co=[float(p) for p in input().split()]


print("Enter the (y2,x2) coordinate")
co1=[float(p2) for p2 in input().split()]



print("Enter the step size of y, then x")
s=2
step=[float(s) for s in input().split()]


pos_x=np.arange(co[0],co1[0]+step[0],step[0])
pos_y=np.arange(co[1],co1[1]+step[1],step[1])

        
print(pos_x)
print(pos_y)

position=[]
for i in range(len(pos_y)):
    for j in range(len(pos_x)):
        position.append([pos_x[j],pos_y[i]])



mirror('off') #make sure mirror is off
capture_photo("bg",1)
laser_power.power=0.01   #make sure laser power is down to 0.01 W
print("\nPatterning will be started within 10 seconds\n")
time.sleep(10)

mirror('off')


xps.move_stage('XYZ.X',position[0][0])
xps.move_stage('XYZ.Y',position[0][1])




vv=[]
#time.sleep(40)
print("\nPatterning Starts\n")
fitdata=[]
for i in range(4,5):
    import csv
    import pandas as pd
    import math
      
    data2=pd.read_csv("dataset.csv")
    rto=data2["ratio"]
    y_unique=[]
    for m in rto:
        if m!=0 and m not in y_unique and  math.isnan(m)!=1:y_unique.append(m)

    number_of_unique_ratios=len(y_unique)
    
   
    
    if number_of_unique_ratios==6:print("AI has started thinking !")
    if i>=number_of_unique_ratios and i>=6:
        import rpy2.robjects as robjects
        robjects.r('''
        sourin<-function(s){
  
        if(s==1){
        require(XML)
        
          suppressWarnings({suppressMessages({
            library(mlrMBO)
            library(ggplot2)
            library(doMC)
          })})

          ps = makeParamSet(
            makeIntegerParam("power", lower = 151, upper =5000),
            makeIntegerParam("time", lower = 500, upper = 10000),
            makeIntegerParam("pressure", lower = 100, upper = 1000)
            
          )

          ctrl = makeMBOControl(y.name = "ratio")
          ctrl = setMBOControlInfill(ctrl, opt = "focussearch",opt.focussearch.maxit = 10, opt.focussearch.points = 10000, crit =makeMBOInfillCritCB(cb.lambda=4))
        
        #dataset-2 should be replaced and later this line as well
          data=read.csv("dataset.csv")
        
        data<-na.omit(data)
          suppressMessages({opt.state = initSMBO(par.set = ps,design = data, control = ctrl, minimize = FALSE, noisy = TRUE)})
          
          print("Proposed parameters:")
          prop <- suppressWarnings({proposePoints(opt.state)})
          print(prop$prop.points) 
          print(prop$crit.components)
          saveRDS(prop,'./prop.rds')
          saveRDS(opt.state,'./opt.state.rds')
          dummy_ratio=0
          cat("Expected value (upper bound):\n")
          cat(paste(prop$crit.components$mean, " (", \
          prop$crit.components$mean + prop$crit.components$se, ")\n", sep = ""))
          
          x<-c(prop$prop.points,dummy_ratio,prop$crit.components$mean,\
          prop$crit.components$mean + prop$crit.components$se)
          
          x2<-c(prop$prop.points)
          #data_plot=read.csv("plot_data.csv")
          
          
          #dataset-2 should be replaced and later this line as well
          write.table(x2, file = "dataset.csv", sep = ",",\
          append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)
          
          #write.table(x, file = "plot_data.csv", sep = ",",\
          #append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)
        
        }
        else{
          prop=readRDS('./prop.rds')
          opt.state=readRDS('./opt.state.rds')
          library(mlrMBO)
          print(prop$crit.components)
          print(prop$prop.points)
          print("Proposed parameters:")
          updateSMBO(opt.state, x = prop$prop.points, y = s)
          prop <- suppressWarnings({proposePoints(opt.state)})
          print(prop$prop.points) 
          print(prop$crit.components)
          saveRDS(prop,'./prop.rds')
          saveRDS(opt.state,'./opt.state.rds')
          dummy_ratio=0
          cat("Expected value (upper bound):\n")
          cat(paste(prop$crit.components$mean, " (", prop$crit.components$mean + prop$crit.components$se, ")\n", sep = ""))
          x<-c(prop$prop.points,dummy_ratio,prop$crit.components$mean,prop$crit.components$mean + prop$crit.components$se)
          x2<-c(prop$prop.points)
          #data_plot=read.csv("plot_data.csv")
          
          #dataset-2 should be replaced and later this line as well
          write.table(x2, file = "dataset.csv", sep = ",", append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)
          #write.table(x, file = "plot_data.csv", sep = ",", append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)
        
        }}

        ''')
        if i==6 and number_of_unique_ratios>=3:
            rsum=robjects.r['sourin']
            rsum((1))
        
        
      
    df=pd.read_csv("dataset.csv")
    power_of_file=df["power"]
    time_of_file=df["time"]
    pressure_of_file=df['pressure']
    xp=position[i][0]
    yp=position[i][1]
    
    xps.move_stage('XYZ.X',xp)
    xps.move_stage('XYZ.Y',yp)
    for j in range(3):
        gopr(pressure_of_file[i])
        close_all()
        time.sleep(5)
    print("\n Pressure is now : ",pressure_of_file[i],"\n")
    
    time.sleep(10)
    

    v=adjust()

    ###begin pre raman spectra check of GO
    
    mirror("on")
    laser_power.power=0.01
    GD=capture_photo("first",i)
    
    mirror('off')
    ##Stops
    if float(GD)<.5:
        print("Moving to next point")
    else:
        a=power_of_file[i]
        
        print("\n")
        print("The power is now: ",a,"\n")
        laser_power.power=((a-2.7979)/0.4639)/1000
        #laser_power.power=((a+10.719)/0.3355)/1000
        time.sleep(30)
        t=time_of_file[i] 
        laser_time=float(t/1e3)
        print("\n")
        print("The time is now: ",laser_time,"\n")
        mirror('on') #irradiation begins

        time.sleep(laser_time+1) #exposure time

        mirror('off')
        time.sleep(5)
        laser_power.power=0.01
        time.sleep(4)

        #begin capturing spectra
        mirror("on")
        import time
   
        p=capture_photo("on",i) 
        #fitdata.append([p,twoGD,twoD,G,WD,WG])
        print("G/D ratio, 2D ratio are : ",p)
        print("\n")
        if i>=number_of_unique_ratios and number_of_unique_ratios>=6:
            if p is None:rsum((0))
                
            else:
                rsum((p[0]))

        mirror("off")
        time.sleep(5)
    
        laser_power.power=.01
 




## G.The adjust() function autofocuses the z-axis each time before patterning a new spot.

In [ ]:

def adjust():
    
    import timeit

    start = timeit.default_timer()
    import time

    zz=[8.5,8.6,8.7,8.8,8.9,9,9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8]
    intensity=[]
    for i in range(len(zz)):

        xps.move_stage('XYZ.Z',zz[i])
        time.sleep(5)
        p=capture_photo('adjust',i)
        intensity.append(p)
    mirror('off')
    stop = timeit.default_timer()

    print('Time: ', stop - start)  
    print(intensity)

    import numpy as np
    ix=intensity.index(np.max(intensity))
    xps.move_stage('XYZ.Z',zz[ix])
    print("adjusted z position is: ",zz[ix])
    return intensity

**The following codes are some random stuff**

In [ ]:
#print(x.position,y.position,z.position)
#from spectra import capture_photo
p=adjust()
pos_all()

In [ ]:
mirror('off')

In [ ]:
#mirror("off")
xps.move_stage('XYZ.X',9.2)
xps.move_stage('XYZ.Y',9.8)
xps.move_stage('XYZ.Z',9.2)

In [ ]:
pos_all()

In [ ]:
xp=position[1][0]
yp=position[1][1]

xps.move_stage('XYZ.X',xp)
xps.move_stage('XYZ.Y',yp)
pos_all()

In [ ]:
mirror('on')

In [ ]:
current_pressure()